In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lsst.daf.persistence as dp

In [ ]:
butler = dp.Butler('/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output')

In [ ]:
def read_tract(butler, tract, filter_='i', num_patches=None):
    if num_patches is None:
        num_patches = len(tract)
    df_list = []
    i = 0
    for patch in tract:
        patchId = '%d,%d' % patch.getIndex()
        dataId = dict(filter=filter_, tract=tract.getId(), patch=patchId)
        try:
            forced = butler.get('deepCoadd_forced_src', dataId)
            calib = butler.get('deepCoadd_calexp_calib', dataId)
            calib.setThrowOnNegativeFlux(False)
            merged = butler.get('deepCoadd_ref', dataId)
        except dp.NoResults as eobj:
            print(eobj)
            continue
        data = merged.asAstropy().to_pandas()
        data[filter_ + '_modelfit_CModel_flux'] = forced['modelfit_CModel_flux']
        data[filter_ + '_modelfit_CModel_fluxSigma'] = forced['modelfit_CModel_fluxSigma']
        data[filter_ + '_mag_CModel'] = calib.getMagnitude(forced['modelfit_CModel_flux'])
        data[filter_ + '_mag_err_CModel'] = calib.getMagnitude(forced['modelfit_CModel_fluxSigma'])
        data[filter_ + '_modelfit_CModel_SNR'] \
            = forced['modelfit_CModel_flux']/forced['modelfit_CModel_fluxSigma']
        data['ext_shapeHSM_HsmShapeRegauss_abs_e'] \
            = np.hypot(data['ext_shapeHSM_HsmShapeRegauss_e1'],
                       data['ext_shapeHSM_HsmShapeRegauss_e2'])
        print(i, patchId, len(data))
        df_list.append(data)
        i += 1
        if i == num_patches:
            break
    return pd.concat(df_list)

In [ ]:
skymap = butler.get('deepCoadd_skyMap')
tractId = 4850
filter_ = 'i'
num_patches = None
data = read_tract(butler, skymap[tractId], filter_=filter_, num_patches=num_patches)
len(data)

In [ ]:
base_mask = ~(np.isnan(data['i_modelfit_CModel_flux'])
         | np.isnan(data['ext_shapeHSM_HsmShapeRegauss_resolution'])
         | np.isnan(data['ext_shapeHSM_HsmShapeRegauss_e1']))
data = data[base_mask]
len(data)

In [ ]:
mask = data['detect_isPrimary']
mask &= data['deblend_skipped'] == False
mask &= data['base_PixelFlags_flag_edge'] == False
mask &= data['base_PixelFlags_flag_interpolatedCenter'] == False
mask &= data['base_PixelFlags_flag_saturatedCenter'] == False
mask &= data['base_PixelFlags_flag_crCenter'] == False
mask &= data['base_PixelFlags_flag_bad'] == False
mask &= data['base_PixelFlags_flag_suspectCenter'] == False
mask &= data['base_PixelFlags_flag_clipped'] == False
mask &= data['ext_shapeHSM_HsmShapeRegauss_flag'] == False

# Cut on object properties
mask &= data['i_modelfit_CModel_SNR'] >= 10
mask &= data['ext_shapeHSM_HsmShapeRegauss_resolution'] >= 0.3
mask &= data['ext_shapeHSM_HsmShapeRegauss_abs_e'] < 2
mask &= data['ext_shapeHSM_HsmShapeRegauss_sigma'] <= 0.4
mask &= data['i_mag_CModel'] < 24.5 # !!! Doesnt have exinction correction
mask &= data['base_Blendedness_abs_flux'] < 10**(-0.375)

In [ ]:
fig = plt.figure(figsize=(10, 10))
p1 = fig.add_subplot(2, 2, 1)
hist_kwds = dict(bins=100, histtype='step')
plt.hist(data['i_mag_CModel'][mask], **hist_kwds);
plt.xlabel('CModel i-mag')
p2 = fig.add_subplot(2, 2, 2)
plt.hist(data['i_modelfit_CModel_SNR'][mask], range=(0, 100), **hist_kwds);
plt.xlabel('CModel i-band S/N')
p3 = fig.add_subplot(2, 2, 3)
plt.hist(data['ext_shapeHSM_HsmShapeRegauss_resolution'][mask], **hist_kwds);
plt.xlabel('HSM resolution')
p4 = fig.add_subplot(2, 2, 4)
plt.hist(data['ext_shapeHSM_HsmShapeRegauss_abs_e'][mask], **hist_kwds);
plt.xlabel('HSM |e|');